### 1. Implement the elastic net TSP by minimizing the following objective function.

$E_{TSP}(P, Y) = \sum_{i = 1}^{N}{\sum_{a = 1}^M{P_{ia}||x_i - y_a||^2}} + \frac{\kappa}{2}\sum_{a = 1}^M{||y_a - y_{a \oplus 1}||_2^2}$

In [3]:
import autograd.numpy as np

In [77]:
from us_city import load_us_cities
US_CITIES = load_us_cities()

DIM = 2 # dimension of the points
N = US_CITIES.shape[0]  # number of cities
M = 3 * N  # number of intermediate points between the cities

KAPPA_START = 20  # starting value of kappa
GAMMA = 1.05  # kappa damping factor

BETA = 10 # inverse temperature

NUM_ITERS = 2 # number of iterations of the algorithm

PERTURBATION_RADIUS = 100 # perturbation radius for generating the intermediate points

**Implement a "driver" function to solve for $Y$ and update $P$ in alternating steps.**

In [85]:
import functools

class Solver(object):

    def __init__(self):
        self.kappa = KAPPA_START

        self.X = US_CITIES
        self.Y = self.init_Y(strategy="centroid", X=self.X, M=M, perturbation_radius=PERTURBATION_RADIUS)
        self.P = np.zeros((N, M))
        print(self.P)

    
    def init_Y(self, strategy="centroid", **kwargs):
        if strategy == "centroid":
            X, M, radius = kwargs["X"], kwargs["M"], kwargs["perturbation_radius"]
            Y = np.tile(
                np.mean(X, axis=0),
                M
            ).reshape((M, DIM))
            # Create perturbation vectors of magnitude radius
            perturbation_vectors = np.random.random((M, DIM))
            perturbation_vectors /= np.linalg.norm(perturbation_vectors, axis=1, ord=2, keepdims=True)
            perturbation_vectors *= radius
            return Y + perturbation_vectors
        raise RuntimeError(f"Unknown strategy: {strategy}")


    @functools.cached_property
    def L(self):
        def L_ij(i, j, M):
            if i == j:
                return 2
            if i == (j + 1) % M:
                return 1
            if j == (i + 1) % M:
                return 1
            return 0
        return np.fromfunction(np.vectorize(L_ij), (M, M), M=M, dtype=int)
    
    
    def get_D(self):
        def d_ij(i, j):            
            if i == j:
                return np.sum(self.P[:, int(j)])
            return 0
        return np.fromfunction(np.vectorize(d_ij), (M, M))


    def solve_for_Y(self):
        """Define solver for Y"""
        # TODO Is there a more computationally efficient way to solve this?
        temp_1 = np.linalg.inv(self.kappa * self.L + self.get_D())
        temp_2 = (self.P.T @ self.X)
        return temp_1 @ temp_2
        
    
    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=0)


    def update_P(self):
        """
        **Define update equation for $P$.**

        We define the elementwise update equation for the matrix $P$:

        $P_{ia} = softmax(-\beta ||x_i - y_a||_2^2)$
        """
        assert self.X.shape[0] == N
        assert self.Y.shape[0] == M
        def p_ij(i, j):
            i, j = int(i), int(j)
            return self.softmax(
                np.linalg.norm(self.X[i] - self.Y[j], ord=2, axis=0)**2
            )
        return np.fromfunction(np.vectorize(p_ij), (N, M))
        

    def run(self):
        for _ in range(NUM_ITERS):
            self.Y = self.solve_for_Y()
            self.P = self.update_P()

In [86]:
s = Solver()
s.run()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/home/msamogh/.cache/pypoetry/virtualenvs/tsp-xdxCQgoB-py3.8/lib/python3.8/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)
/tmp/ipykernel_9720/1081775070.py:59: RuntimeWarning: invalid value encountered in double_scalars
  return np.exp(x) / np.sum(np.exp(x), axis=0)
/home/msamogh/.cache/pypoetry/virtualenvs/tsp-xdxCQgoB-py3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2387: RuntimeWarning: invalid value encountered in p_ij (vectorized)
  outputs = ufunc(*inputs)


In [87]:
s.P

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])